In [3]:
!pip install pyspark
!pip install pandas
!pip install findspark

You should consider upgrading via the 'c:\users\acer\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\acer\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


  Using cached findspark-2.0.1-py2.py3-none-any.whl (4.4 kB)


You should consider upgrading via the 'c:\users\acer\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [4]:
import pyspark
import pandas as pd

import findspark
findspark.init()

In [5]:
data = pd.read_csv("./ds_salaries.csv")
data.head(10)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M
5,2023,SE,FT,Applied Scientist,222200,USD,222200,US,0,US,L
6,2023,SE,FT,Applied Scientist,136000,USD,136000,US,0,US,L
7,2023,SE,FT,Data Scientist,219000,USD,219000,CA,0,CA,M
8,2023,SE,FT,Data Scientist,141000,USD,141000,CA,0,CA,M
9,2023,SE,FT,Data Scientist,147100,USD,147100,US,0,US,M


In [6]:
from pyspark.sql import SparkSession
from pyspark import SparkContext


### Create Session

In [7]:
spark = SparkSession.builder.appName('Assignment').getOrCreate()

### Create context

In [8]:
sc = spark.sparkContext

In [9]:
spark

In [10]:
df_pyspark = spark.read.csv("./ds_salaries.csv");
df_pyspark.show()

+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|      _c0|             _c1|            _c2|                 _c3|   _c4|            _c5|          _c6|               _c7|         _c8|             _c9|        _c10|
+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
|     2023|              SE|             FT|Principal Data Sc...| 80000|            EUR|        85847|                ES|         100|              ES|           L|
|     2023|              MI|             CT|         ML Engineer| 30000|            USD|        30000|                US|         100|              US|           S|
|     2023

### parallelize example


In [11]:
data = [1,2,3,4,5]
rdd = sc.parallelize(data)

In [12]:
data = [1,2,3,4,5]
rdd = sc.parallelize(data)
squred_root = rdd.map(lambda x: x*2);
# print("first()", squred_root.first());
print("map():", squred_root.collect())

map(): [2, 4, 6, 8, 10]


### Load dataset with header as column names

In [13]:
spark_data = spark.read.option('header', 'true').csv("./ds_salaries.csv")
spark_data.show()

+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|     2023|              SE|             FT|Principal Data Sc...| 80000|            EUR|        85847|                ES|         100|              ES|           L|
|     2023|              MI|             CT|         ML Engineer| 30000|            USD|        30000|                US|         100|              US|           S|
|     2023|              MI|             CT|         ML Engineer| 25500|            USD|        25500|                US|         100|              US|           S|
|     2023

In [13]:
spark_data.head(10)

[Row(work_year='2023', experience_level='SE', employment_type='FT', job_title='Principal Data Scientist', salary='80000', salary_currency='EUR', salary_in_usd='85847', employee_residence='ES', remote_ratio='100', company_location='ES', company_size='L'),
 Row(work_year='2023', experience_level='MI', employment_type='CT', job_title='ML Engineer', salary='30000', salary_currency='USD', salary_in_usd='30000', employee_residence='US', remote_ratio='100', company_location='US', company_size='S'),
 Row(work_year='2023', experience_level='MI', employment_type='CT', job_title='ML Engineer', salary='25500', salary_currency='USD', salary_in_usd='25500', employee_residence='US', remote_ratio='100', company_location='US', company_size='S'),
 Row(work_year='2023', experience_level='SE', employment_type='FT', job_title='Data Scientist', salary='175000', salary_currency='USD', salary_in_usd='175000', employee_residence='CA', remote_ratio='100', company_location='CA', company_size='M'),
 Row(work_year